In [1]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-20 19:11:13--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 19:11:13 (9.18 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [13]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-27 16:23:43--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.04s   

2024-06-27 16:23:43 (17.9 MB/s) - ‘documents.json’ saved [658332/658332]



In [10]:
import minsearch
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()
from elasticsearch import Elasticsearch
#start elastic search from docker image first
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'a76e50f4ab61', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'C591eCJ_T-yJBqFGpZx1kg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

In [14]:
with open('documents.json', 'r') as f:
    doc_raws = json.load(f)
documents = []
for course_dict in doc_raws:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "module_02_index"
es_client.indices.create(index = index_name, body = index_settings)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [4]:
#client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [17]:
index = minsearch.Index(text_fields = ['question', 'text', 'section'],
                         keyword_fields = ['course']
                        )
index.fit(documents)

In [24]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    result  = index.search(query = query,
                 boost_dict = boost,
                 num_results = 5
                )
    return result

def elastic_search (query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    es_result = es_client.search(index = index_name, body=search_query)
    es_result_docs = []
    for hit in es_result['hits']['hits']:
        es_result_docs.append(hit['_source'])


    return es_result_docs
    
def build_prompt(query, search_result):

    context = ""
    for doc in search_result:
        context =context + f"section :{doc['section']} \nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt_template = """
    You are a course teaching assistant. Answer the given QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}\n
    CONTEXT: {context}
    """.strip()
    prompt = prompt_template.format(question = query, 
                                    context = context).strip()
    return prompt

def llm (prompt, generate_params =None):
    if generate_params is None:
        generate_params= {}
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids #.to("cuda")
    outputs = model.generate(input_ids,
                             max_length = generate_params.get('max_length', 100),
                             num_beams = generate_params.get('num_beams',5),
                             do_sample=generate_params.get("do_sample", False),
                            temperature=generate_params.get("temperature", 1.0),
                            top_k=generate_params.get("top_k", 50),
                            top_p=generate_params.get("top_p", 0.95)
                            )
    result = tokenizer.decode(outputs[0])
    return result

In [25]:
def rag(query):
    results = elastic_search(query)#search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [28]:
query = "The course on llms and rag is awesome, I love it"
rag(query)

'<pad> Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes. You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.</s>'